In [3]:
import numpy as np
import matplotlib.pyplot as plt
import LowerUpperFunctionsDatasets as luf

In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

# --- load iris into a DataFrame ---
iris = load_iris(as_frame=True)
df = iris.frame
df.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]

# X1, X2 ~ sepal_length, sepal_width
X_cols = ["sepal_length", "sepal_width"]
Y_col = "petal_length"   # target for f_values


# --- helper to generate (a,b,c) for one feature from its range ---
def make_feature_params(df, col, n_terms):
    """
    Build n_terms triangular fuzzy sets over the range of df[col].
    Triangles have centers equally spaced between min and max,
    and support [center - k, center + k], where k is the distance
    between neighboring centers.
    """
    x = df[col].values
    xmin, xmax = x.min(), x.max()

    if n_terms == 1:
        center = 0.5 * (xmin + xmax)
        span = (xmax - xmin) or 1.0
        return [(center - span, center, center + span)]

    centers = np.linspace(xmin, xmax, n_terms)
    k = centers[1] - centers[0]

    params = [(c - k, c, c + k) for c in centers]
    return params


# --- define fuzzy partitions for sepal_length (X1) and sepal_width (X2) ---
n1 = 3   # number of triangular sets for sepal_length
n2 = 2   # number of triangular sets for sepal_width

params_X1 = make_feature_params(df, "sepal_length", n1)
params_X2 = make_feature_params(df, "sepal_width", n2)

feature_partitions = {
    "sepal_length": params_X1,   # X1
    "sepal_width": params_X2,    # X2
    # you can add "petal_width": params_X3, etc.
}

# --- your functional part ---
f_values = df[Y_col].values

# assuming your module luf already contains:
#   compute_F_A_up_for_all_combinations(df, feature_partitions, f_values)
#   f_A_upT_multi(x_point, feature_partitions, results)

results = luf.compute_F_A_up_for_all_combinations(
    df[X_cols],        # only the columns used in feature_partitions
    feature_partitions,
    f_values
)

# example point in the iris feature space
x = {"sepal_length": 5.1, "sepal_width": 3.5}
f_hat = luf.f_A_upT_multi(x, feature_partitions, results)

print("Estimated f(x) =", f_hat)


Estimated f(x) = 3.149999999999997


In [6]:
import pandas as pd

table = []
for r in results:
    row = r["terms"].copy()   # columns: which fuzzy term used
    row["F_A_up"] = r["F_A_up"]
    table.append(row)

df_table = pd.DataFrame(table)
print(df_table)


   sepal_length  sepal_width    F_A_up
0             0            0  3.000000
1             0            1  1.750000
2             1            0  4.583333
3             1            1  3.266667
4             2            0  5.175000
5             2            1  5.025000
